**<h1 align="center">Sales Forecasting</h1>**
**<h1 align="center">Siemens</h1>**

**Group members:**<br> 
Chloé Deschanel - 20240693 <br>
Diogo Carvalho - 20240694 <br>
Ingrid Lopez - 20240692 <br>
Ruben Marques- 20240352

# Table of Contents

* [1. Data Exploration](#chapter1)
    * [1.1. Imports](#sub-section-1_1)
    * [1.2. Loading Data](#sub-section-1_2)
    * [1.3. Sales Data](#sub-section-1_3)
        * [1.3.1. Initial Analysis](#sub-section-1_3_1)



        
        * [1.3.2. Missing Values](#sub-section-1_3_2)
        * [1.3.3. Duplicates](#sub-section-1_3_3)
        * [1.3.4. Data Statistics](#sub-section-1_3_4)
    * [1.4. Individual Variable Analysis](#sub-section-1_4)
        * [1.4.1. Numerical Variables](#sub-section-1_4_1)
            * [1.4.1.1. Overview](#sub-section-1_4_1_1)
        * [1.4.2. Categorical Variables](#sub-section-1_4_2)
        * [1.4.3. Binary Variables](#sub-section-1_4_3)


        * [1.3.1. Types](#sub-section-1_3_1)
        * [1.3.2. Missing Values](#sub-section-1_3_2)
        * [1.3.3. Duplicates](#sub-section-1_3_3)
        * [1.3.4. Data Statistics](#sub-section-1_3_4)

<a class="anchor" id="chapter1"></a>

# 1. Data Exploration

</a>

<a class="anchor" id="sub-section-1_1"></a>

## 1.1. Imports

</a>

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Configurations
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Force normal decimal formatting
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
# Visualisations 
sns.set_style("white")
siemens_green = '#009999'
siemens_colors = ['#009999', '#006666', '#00CCCC']

<a class="anchor" id="sub-section-1_2"></a>

## 1.2. Loading Data

</a>

In [ ]:
# Import xlsx dataset into pandas dataframe
market_data = pd.read_excel('../Data/Case2_Market data.xlsx', header=[0,1,2])

# Import csv dataset into pandas dataframe
sales_data = pd.read_csv('../Data/Case2_Sales data.csv', sep=";")

<a class="anchor" id="sub-section-1_3"></a>

## 1.3. Sales Data

</a>

<a class="anchor" id="sub-section-1_3_1"></a>

### 1.3.1. Initial Analysis

</a>